## 라즈베리파이3

+ 부팅해서, `country`, `암호` 설정
+ 리부팅
+ 터미널에서 `sudo raspi-config` 실행
    - SSH와 Camera 활성화

### 고정 아이피 설정

`/etc/dhcpcd.conf` 에 추가 : 명령 `sudo nano /etc/dhcpcd.conf`

```
interface eth0
static ip_address=203.247.198.203/24
static routers=203.247.198.1
static domain_name_servers=168.126.63.1 8.8.8.8
```

지정하고, 리부팅

```
sudo reboot
```

### SSH 접속

+ 유저명@호스트 : `pi@라즈베리파이아이피`
+ 포트 : 22
+ 암호 : 지정하신 암호

### picamera 샘플 코드

```python
# sample_picamera_01.py

import time
from picamera import PiCamera

camera = PiCamera()
camera.start_preview()
print('sleep 3 seconds ...')
time.sleep(3)
print('captured')
camera.capture('capture.jpg')
camera.stop_preview()
camera.close()
```

실행

```
python3 sample_picamera_01.py
```

### 비디오 녹화

```
```

h.264 비디오 파일 재생

```
omxplayer 파일경로
```

### Jupyter 설치

```
pip3 install jupyter
```

현재 jupyter 명령은 `~/.local/bin` 에 저장

그런데, 환경변수 `PATH`에 위 경로가 없어서, jupyter 명령 실행이 안 되고 있음.

현재 쉘에 대한 환경설정 파일에 다음 코드를 추가 : ex) `~/.bashrc`

```sh
export PATH=${HOME}/.local/bin:${PATH}
```

위 코드를 추가하시고, `source ~/.bashrc` 명령을 통해, 재로딩

```sh
jupyter notebook password   # 암호 지정
jupyter notebook --ip=0.0.0.0
```

### 서버에서 winscp를 통한 파일 시스템 접근

https://winscp.net

ssh는 sftp 프로토콜을 지원하고있습니다. 이를 통해 파일/디렉토리를 업로드/다운로드를 수행하실 수 있습니다.

미션

+ 여러분의 라즈베리파이3 상의 `capture.jpg` 파일을 로컬로 복사해보세요.

## AWS Rekognition

IAM -> Add User

raspberry3 이름의 User

 + Programatic Access
 + Permission : AmazonRekognitionFullAccess
 
**Access Key ID**와 **Secret Access Key**를 잘 메모해두세요.

In [14]:
import boto3

# TODO: access key, secret access key, region 지정
import os
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_DEFAULT_REGION'] = 'ap-northeast-1'  # Tokyo

client = boto3.client('rekognition')

In [ ]:
with open("1.jpg", "rb") as f:
    image_data = f.read()

client.detect_labels(Image={'Bytes': image_data})

In [20]:
# 전 멤버에 대해서 각자 사진을 2장씩 (다른 포즈) 찍어서 해보세요.

with open("마동석.jpg", "rb") as f:
    image_data = f.read()

client.detect_faces(Image={'Bytes': image_data}, Attributes=['ALL'])

{'FaceDetails': [{'BoundingBox': {'Width': 0.17555555701255798,
    'Height': 0.263145387172699,
    'Left': 0.4611110985279083,
    'Top': 0.12157982587814331},
   'AgeRange': {'Low': 35, 'High': 52},
   'Smile': {'Value': True, 'Confidence': 65.72525787353516},
   'Eyeglasses': {'Value': False, 'Confidence': 99.81666564941406},
   'Sunglasses': {'Value': False, 'Confidence': 99.83206939697266},
   'Gender': {'Value': 'Male', 'Confidence': 99.92861938476562},
   'Beard': {'Value': False, 'Confidence': 64.11399841308594},
   'Mustache': {'Value': True, 'Confidence': 82.99322509765625},
   'EyesOpen': {'Value': True, 'Confidence': 99.03519439697266},
   'MouthOpen': {'Value': False, 'Confidence': 99.98152923583984},
   'Emotions': [{'Type': 'HAPPY', 'Confidence': 45.8049430847168},
    {'Type': 'CALM', 'Confidence': 7.305383682250977},
    {'Type': 'CONFUSED', 'Confidence': 5.979972839355469}],
   'Landmarks': [{'Type': 'eyeLeft',
     'X': 0.5216698050498962,
     'Y': 0.22411632537841

### 성인 컨텐츠 탐지

```python
response = client.detect_moderation_labels(Image={'Bytes': image_data})
```

### 같은 얼굴 비교

```python
response = client.compare_faces(
    SourceImage={'Bytes': image_data_1},
    TargetImage={'Bytes': image_data_1},
)

matches = response['FaceMatches']
if matches:
    for match in matches:
        similarity = match['Similarity']
        print('유사도: {:.1f}%'.format(similarity))
else:
    print('매칭정보를 찾지 못했습니다.')

response['FaceMatches']
```

## 얼굴 Collection

In [22]:
COLLECTION_ID = 'tongmyong-students-group1'

In [23]:
client.create_collection(CollectionId=COLLECTION_ID)

{'StatusCode': 200,
 'CollectionArn': 'aws:rekognition:ap-northeast-1:095307090881:collection/tongmyong-students-group1',
 'FaceModelVersion': '3.0',
 'ResponseMetadata': {'RequestId': '9608185f-8be3-11e8-85bc-2d07b67721c1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 20 Jul 2018 06:10:21 GMT',
   'x-amzn-requestid': '9608185f-8be3-11e8-85bc-2d07b67721c1',
   'content-length': '142',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [25]:
# FIXME: 각 학생 별로 아이디/사진 dict/list를 구성해주세요.
# Key는 영어로만 지정해주세요.

member_dict = {
    'blackpink-rose': [
        'faces/블랙핑크-로제-1.jpg',
        'faces/블랙핑크-로제-2.jpg',
    ],
    'blackpink-lisa': [
        'faces/블랙핑크-리사-1.jpg',
        'faces/블랙핑크-리사-2.jpg',
    ],
    'blackpink-jeny': [
        'faces/블랙핑크-제니-1.jpg',
        'faces/블랙핑크-제니-2.jpg',
    ],
}

In [26]:
for (user_id, image_list) in member_dict.items():
    print(user_id, image_list)

blackpink-rose ['faces/블랙핑크-로제-1.jpg', 'faces/블랙핑크-로제-2.jpg']
blackpink-lisa ['faces/블랙핑크-리사-1.jpg', 'faces/블랙핑크-리사-2.jpg']
blackpink-jeny ['faces/블랙핑크-제니-1.jpg', 'faces/블랙핑크-제니-2.jpg']


In [28]:
for (user_id, image_list) in member_dict.items():
    for image_path in image_list:
        print('indexing {} ...'.format(image_path))
        with open(image_path, 'rb') as f:
            image_data = f.read()

        client.index_faces(
            CollectionId=COLLECTION_ID,
            ExternalImageId=user_id,
            Image={'Bytes': image_data},
            DetectionAttributes=['ALL'])

print('completed!!!')

blackpink-rose ['faces/블랙핑크-로제-1.jpg', 'faces/블랙핑크-로제-2.jpg']
indexing faces/블랙핑크-로제-1.jpg ...
indexing faces/블랙핑크-로제-2.jpg ...
blackpink-lisa ['faces/블랙핑크-리사-1.jpg', 'faces/블랙핑크-리사-2.jpg']
indexing faces/블랙핑크-리사-1.jpg ...
indexing faces/블랙핑크-리사-2.jpg ...
blackpink-jeny ['faces/블랙핑크-제니-1.jpg', 'faces/블랙핑크-제니-2.jpg']
indexing faces/블랙핑크-제니-1.jpg ...
indexing faces/블랙핑크-제니-2.jpg ...
completed!!!


In [30]:
with open("faces/누구야.jpg", "rb") as f:
    image_data = f.read()

response = client.search_faces_by_image(
    CollectionId=COLLECTION_ID,
    Image={'Bytes': image_data},
)
matches = response['FaceMatches']
matches

[{'Similarity': 96.24583435058594,
  'Face': {'FaceId': 'f8a2b9bd-07ae-40a8-a08d-e93a94d58eaa',
   'BoundingBox': {'Width': 0.40384599566459656,
    'Height': 0.40384599566459656,
    'Left': 0.2961539924144745,
    'Top': 0.06153849884867668},
   'ImageId': '3965b545-c85d-5fa7-9fd3-04588e7b68d7',
   'ExternalImageId': 'blackpink-lisa',
   'Confidence': 99.71879577636719}},
 {'Similarity': 94.80065155029297,
  'Face': {'FaceId': '4cf2c52a-368f-4af5-b77c-ed687e813b44',
   'BoundingBox': {'Width': 0.30156201124191284,
    'Height': 0.20104199647903442,
    'Left': 0.30937498807907104,
    'Top': 0.10312499850988388},
   'ImageId': 'c5ef1331-bb97-56e1-9dcf-ae84b961a5a6',
   'ExternalImageId': 'blackpink-lisa',
   'Confidence': 99.9572982788086}}]

In [33]:
for face_meta in matches:
    print('{}의 확률로 {}입니다.'.format(
        face_meta['Face']['Confidence'],
        face_meta['Face']['ExternalImageId']
    ))

99.71879577636719의 확률로 blackpink-lisa입니다.
99.9572982788086의 확률로 blackpink-lisa입니다.
